In [109]:
!python precoss_dataset.py ../data/01_data_processed ../data/01_data_processed

Graph(num_nodes={'user': 1970, 'wifi': 15336},
      num_edges={('user', 'relation', 'wifi'): 15405, ('wifi', 'relation-by', 'user'): 15405},
      metagraph=[('user', 'wifi', 'relation'), ('wifi', 'user', 'relation-by')])
{'weight': tensor([1., 1., 1.,  ..., 1., 1., 1.])}
Done!


Using backend: pytorch


In [81]:
import numpy as np
import pandas as pd
import os

In [86]:
directory = '../data/01_data_processed'
users = pd.read_csv(os.path.join(directory,'user.csv'))
users = users[['customer_id','trans_time','target']]
users

,customer_id,trans_time,target
0,862201,2020-07-13 18:05:08,1
1,4271399,2020-07-27 10:25:27,1
2,4284127,2020-08-01 09:15:17,1
3,4237165,2020-07-03 13:05:09,0
4,4091456,2020-06-12 19:05:11,0
...,...,...,...
12102,4235548,2020-07-17 11:35:05,1
12103,4290699,2020-08-05 21:40:08,1
12104,4281305,2020-07-30 03:20:02,1
12105,4268429,2020-07-21 16:05:05,1


In [87]:
users.trans_time.min(),users.trans_time.max()

('2020-06-01 01:00:43', '2020-08-06 21:35:06')

In [88]:
users.target.value_counts().loc[0]/users.shape[0]

0.13603700338647062

In [91]:
wifi_list = pd.read_csv(os.path.join(directory,'wifi_list_id.csv'))
wifi_list = wifi_list[['customer_id','create_time','wifi_list']]
wifi_list = wifi_list.sort_values(['customer_id','wifi_list','create_time'])
wifi_list = wifi_list.drop_duplicates(['customer_id','wifi_list'])
wifi_list = wifi_list.rename(columns={'wifi_list':'wifi_id'})
wifi_list

,customer_id,create_time,wifi_id
25830,107056,2020-09-04 16:44:55,00:67:62:6d:02:e8
38719,107056,2020-09-08 19:46:41,2e:ff:ee:92:a8:03
38720,107056,2020-09-08 19:46:41,74:da:88:38:7f:2a
25826,107056,2020-09-04 16:44:55,80:26:89:58:d9:6b
25823,107056,2020-09-04 16:44:55,90:f6:52:ac:c3:c2
...,...,...,...
19207,4410632,2020-09-23 05:38:49,e8:65:d4:36:ec:30
19206,4410632,2020-09-23 05:38:49,f0:b4:d2:81:30:e6
19199,4410632,2020-09-23 05:38:49,f4:1d:6b:a5:88:70
19202,4410632,2020-09-23 05:38:49,fc:a6:cd:96:cd:f8


In [92]:
user_wifi_list = pd.merge(users,wifi_list,how='inner',on='customer_id')
user_wifi_list.shape

(15405, 5)

In [93]:
user_wifi_list[user_wifi_list['trans_time']>user_wifi_list['create_time']].shape

(15405, 5)

In [94]:
user_wifi_list

,customer_id,trans_time,target,create_time,wifi_id
0,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,0c:98:38:a9:f0:6d
1,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,0e:a8:a7:5d:ca:34
2,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,74:5b:c5:43:1f:9c
3,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,9a:c8:b8:90:5c:9b
4,4202015,2020-06-27 06:35:10,1,2020-06-24 16:22:09,f0:8c:fb:29:13:f0
...,...,...,...,...,...
15400,4235548,2020-07-17 11:35:05,1,2020-07-03 09:53:07,a0:ab:1b:70:ed:1c
15401,4235548,2020-07-17 11:35:05,1,2020-07-16 12:03:21,b4:e9:b0:9e:95:40
15402,4235548,2020-07-17 11:35:05,1,2020-07-16 12:03:21,b4:e9:b0:9e:95:4e
15403,4235548,2020-07-17 11:35:05,1,2020-07-16 12:03:21,b4:e9:b0:9e:95:4f


In [95]:
distinct_users_in_relation = wifi_list['customer_id'].unique()
distinct_wifi_in_relation = wifi_list['wifi_id'].unique()
users = users[users['customer_id'].isin(distinct_users_in_relation)]
wifi_list = wifi_list[wifi_list['wifi_id'].isin(distinct_wifi_in_relation)]

In [97]:
users.shape

(1970, 3)

In [98]:
users.target.value_counts()

1    1713
0     257
Name: target, dtype: int64

In [99]:
users.target.value_counts().loc[0]/users.shape[0]

0.1304568527918782

In [59]:
wifi_list.shape

(58262, 3)

In [70]:
wifi_list.drop_duplicates('wifi_id').shape

(57620, 3)

In [71]:
wifi_list

,customer_id,create_time,wifi_id
25830,107056,2020-09-04 16:44:55,00:67:62:6d:02:e8
38719,107056,2020-09-08 19:46:41,2e:ff:ee:92:a8:03
38720,107056,2020-09-08 19:46:41,74:da:88:38:7f:2a
25826,107056,2020-09-04 16:44:55,80:26:89:58:d9:6b
25823,107056,2020-09-04 16:44:55,90:f6:52:ac:c3:c2
...,...,...,...
19207,4410632,2020-09-23 05:38:49,e8:65:d4:36:ec:30
19206,4410632,2020-09-23 05:38:49,f0:b4:d2:81:30:e6
19199,4410632,2020-09-23 05:38:49,f4:1d:6b:a5:88:70
19202,4410632,2020-09-23 05:38:49,fc:a6:cd:96:cd:f8


In [61]:
entities = users['customer_id'].astype('category')
print(entities)
if not (entities.value_counts() == 1).all():
    raise ValueError('Different entity with the same primary key detected.')

6        4202015
16       4291583
22        880193
23       4148588
34       4220955
          ...   
12079    4121737
12084    4119054
12088    1315610
12091    4286858
12102    4235548
Name: customer_id, Length: 1970, dtype: category
Categories (1970, int64): [152917, 160937, 173091, 174326, ..., 4291897, 4291937, 4291980, 4291982]


In [69]:
entities = wifi_list['wifi_id'].astype('category')
print(entities)
if not (entities.value_counts() == 1).all():
    raise ValueError('Different entity with the same primary key detected.')

25830    00:67:62:6d:02:e8
38719    2e:ff:ee:92:a8:03
38720    74:da:88:38:7f:2a
25826    80:26:89:58:d9:6b
25823    90:f6:52:ac:c3:c2
               ...        
19207    e8:65:d4:36:ec:30
19206    f0:b4:d2:81:30:e6
19199    f4:1d:6b:a5:88:70
19202    fc:a6:cd:96:cd:f8
19198    fc:a6:cd:96:e5:e0
Name: wifi_id, Length: 58262, dtype: category
Categories (57620, object): [00:00:00:00:00:00, 00:01:23:01:4c:d0, 00:01:23:05:7e:08, 00:01:23:05:7f:a0, ..., fe:ec:da:f2:5b:f3, fe:ec:da:f2:5e:9f, fe:ec:da:f2:9f:17, fe:f9:d5:35:d9:5d]


ValueError: Different entity with the same primary key detected.